Unfortunately, there are some workarounds required.
```
ln -s vtkm/thirdparty/diy/vtkmdiy/ vtkmdiy
ln -s vtkm/thirdparty/taotuple/vtkmtaotuple/ vtkmtaotuple
```

in the 

```
include/vtkm-1.2
```

installation directory. These are normally handled by the `VTKmTargets.cmake` in

```
lib/cmake/vtkm-1.2/
```


In [1]:
//#pragma include_alias("/local/vtk-m/clang/include/vtkm-1.2/vtkm/thirdparty/taotuple/vtkmtaotuple/","vtkmtaotuple")
#include <lodepng_config_cling.h>
#include <vtkm_config_cling.h>

In [2]:
#include <vtkm/cont/DeviceAdapter.h>
#include <vtkm/cont/testing/MakeTestDataSet.h>
#include <vtkm/cont/testing/Testing.h>
#include <vtkm/rendering/Actor.h>
#include <vtkm/rendering/CanvasRayTracer.h>
#include <vtkm/rendering/MapperRayTracer.h>
#include <vtkm/rendering/Scene.h>
#include <vtkm/rendering/View3D.h>
#include <vtkm/rendering/testing/RenderTest.h>
#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"
#include "xcpp/xdisplay.hpp"

In [3]:
vtkm::Float64 t, eps, omega;
t = eps = omega = 0.0;
std::vector<std::vector<vtkm::Float64>> linesx;
std::vector<std::vector<vtkm::Float64>> linesy;
int xdens, ydens;
xdens = ydens = 16;
std::unique_ptr<xpl::figure> fig = std::unique_ptr<xpl::figure>(new xpl::figure());


Use lodepng to save out png instead of pnm so jupyter can read it.


In [4]:
#include <vtkm/Math.h>
//anonymous lambda as a work around for redefinition when recompiling
    
using Vec2 = vtkm::Vec<vtkm::Float64,2>;

//     t = 0.5;
//     eps = 0.25;
//     omega = 0.25;
auto a = [&](){
    return eps * vtkm::Sin(omega*t);
};
auto b = [&](){
    return 1 -2 * eps * vtkm::Sin(omega * t);
};
auto f = [&](auto x){
  return a() *x*x + b() * x;  
};
auto calcU = [&f](auto x, auto y, auto A){
    return -vtkm::Pi() * A * vtkm::Sin(vtkm::Pi() * f(x)) * vtkm::Cos(vtkm::Pi() * y);
};
auto calcV = [&](auto x, auto y, auto A){
    return vtkm::Pi() * A * vtkm::Cos(vtkm::Pi() * f(x)) * vtkm::Sin(vtkm::Pi()*y) * (2 * a() * x + b());
};

auto doublegyre = [&](auto x, auto y){
    auto A = 0.25;
    eps = 0.25;
    omega = 2.0 * vtkm::Pi() / 10.0;

    auto u = calcU(x,y, A);
    auto v = calcV(x,y, A);
    return std::make_tuple(u,v);
};
t = 0.5;
doublegyre(0.25,0.25);

        


@0x9b00650

In [5]:
[](){
  auto try_and_error = [](){
      t = 0.1;

      xdens = 16;
      ydens = 16;
      linesx.resize(xdens*ydens);
      linesy.resize(xdens*ydens);
      
      int x,y;
      x = y = 0;
    for (y=0; y<ydens; y++){
        for (x=0; x < xdens; x++){
            int idx = y*xdens+x;
              linesx[idx].clear();
              linesy[idx].clear();
            vtkm::Float32 xpt = x/float(xdens);
            vtkm::Float32 ypt = y/float(ydens);
            vtkm::Float32 newx,newy;
            std::tie(newx,newy) = doublegyre(xpt,ypt);
            linesx[idx].push_back(xpt);
            linesy[idx].push_back(ypt);
            linesx[idx].push_back(xpt + newx/float(xdens));
            linesy[idx].push_back(ypt + newy/float(ydens));
        }
    }
  } ;
  try_and_error();
}();


In [6]:
xpl::linear_scale sx, sy;

In [7]:
xpl::axis hx(sx), hy(sy);
hy.orientation = "vertical";
fig->add_axis(hx);
fig->add_axis(hy);



In [16]:
fig = std::unique_ptr<xpl::figure>(new xpl::figure());
fig->add_axis(hx);
fig->add_axis(hy);
for (int i=3; i<4;i++){
    for (int j=3; j<4; j++){
    xpl::lines line(sx,sy);
    line.x = linesx[i * xdens + j];
    line.y = linesy[i * xdens + j];
    std::cout << linesy[i*xdens +j][0] << std::endl;
    fig->add_mark(std::move(line));
        
    }
    
}
*fig

0.1875


A Jupyter widget